In [2]:
import tkinter as tk
from tkinter import *
import customtkinter
from PIL import Image, ImageTk
from tkinter import messagebox
import cv2
import os
import numpy as np
from keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")
import keras
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import  load_model
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


customtkinter.set_appearance_mode("dark")
customtkinter.set_default_color_theme("dark-blue")


root = customtkinter.CTk()
root.geometry('900x600+350+100')
root.title('Human Emotion Detector')
root.configure(bg = '#CCCCCC')
root.resizable(height = False, width = False)



# Define Images

bg = ImageTk.PhotoImage(file = 'emotions.png')

add_start_image = ImageTk.PhotoImage(Image.open('start.png').resize((20, 20), Image.ANTIALIAS))

# Labels

bl1 = Label(root, image = bg)
bl1.place(x = 0, y = 0, relwidth = 1, relheight = 1)


l = Label(root, text = 'Human Emotion Detector', bg = '#CCCCCC', width = 20,
        bd =0, font ='timesnewroman 21 bold italic underline', fg ='#999999')
l.place(x = 350, y = 0)


# Button Function

def started():
    
    messagebox.showinfo('Starting','Starting')
    
    # load model
    model = load_model("emotion_model.h5")


    face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


    cap = cv2.VideoCapture(0)

    while True:
        ret, test_img = cap.read()  # captures frame and returns boolean value and captured image
        if not ret:
            continue
        gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

        faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

        for (x, y, w, h) in faces_detected:
            cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=7)
            roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
            roi_gray = cv2.resize(roi_gray, (224, 224))
            img_pixels =tf.keras.preprocessing.image.img_to_array(roi_gray)
            img_pixels = np.expand_dims(img_pixels, axis=0)
            img_pixels /= 255

            predictions = model.predict(img_pixels)

            # find max indexed array
            max_index = np.argmax(predictions[0])

            emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
            predicted_emotion = emotions[max_index]

            cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        resized_img = cv2.resize(test_img, (1000, 700))
        cv2.imshow('Facial emotion analysis ', resized_img)

        if cv2.waitKey(10) == ord('q'):  # wait until 'q' key is pressed
            break

    cap.release()
    cv2.destroyAllWindows



# Create Button

button_1 = customtkinter.CTkButton(master = root, image = add_start_image, text = 'Start', width = 120, height = 40, compound = 'left',
                                  fg_color = '#999999', hover_color = '#CCCCCC', command = started)
button_1.pack(side = 'bottom', pady = 55, padx = 50)

root.mainloop()